In [1]:
import sys

sys.path.insert(0, '..')

In [2]:
from morpheus.project.application.write import project_command_bus
from morpheus.project.application.read.ModelReader import ModelReader

## Setup the Rio Primero project

In [3]:
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.Project import CreateProjectCommand, CreateProjectCommandHandler

user_id = UserId.new()
project_id = ProjectId.new()

create_project_command = CreateProjectCommand(
  project_id=project_id,
  name=Name('Rio Primero'),
  description=Description('Rio Primero Project in Argentina'),
  tags=Tags.from_list(['rio primero', 'argentina']),
  user_id=user_id
)

project_command_bus.dispatch(create_project_command)

# Create Model

In [4]:
from morpheus.project.types.discretization.spatial import Rotation
from morpheus.project.types.geometry import Polygon
from morpheus.project.types.Model import ModelId
from morpheus.project.application.write.Model import CreateModelCommand

model_id = ModelId.new()

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

create_model_command = CreateModelCommand(
  project_id=project_id,
  user_id=user_id,
  model_id=model_id,
  geometry=polygon,
  n_cols=100,
  n_rows=50,
  rotation=Rotation.from_float(0.0),
)

project_command_bus.dispatch(create_model_command)

## Setup Time Discretization

In [5]:
from datetime import datetime
from morpheus.project.application.write.Model import UpdateModelTimeDiscretizationCommand
from morpheus.project.types.discretization.time import TimeDiscretization
from morpheus.project.types.discretization.time.TimeUnit import TimeUnit
from morpheus.project.types.discretization.time.Stressperiods import StartDateTime, StressPeriodCollection, \
  StressPeriod, NumberOfTimeSteps, TimeStepMultiplier, IsSteadyState

time_discretization = TimeDiscretization(
  start_date_time=StartDateTime.from_datetime(datetime(2015, 1, 1)),
  end_date_time=StartDateTime.from_datetime(datetime(2020, 12, 31)),
  stress_periods=StressPeriodCollection([
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2015, 1, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.yes()
    ),
  ]),
  time_unit=TimeUnit.days()
)

update_model_time_discretization_command = UpdateModelTimeDiscretizationCommand(
  project_id=project_id,
  time_discretization=time_discretization,
  user_id=user_id,
)

project_command_bus.dispatch(update_model_time_discretization_command)

## Create Top Layer

In [6]:
from morpheus.project.types.layers.Layer import LayerId, LayerName, LayerProperties, LayerDescription, LayerType
from morpheus.project.application.write.Model import CreateModelLayerCommand, UpdateModelLayerCommand

create_model_layer_command = CreateModelLayerCommand(
  project_id=project_id,
  model_id=model_id,
  user_id=user_id,
  layer_id=LayerId.new(),
  name=LayerName('Top Layer'),
  type=LayerType.convertible(),
  description=LayerDescription('Top Layer'),
  properties=LayerProperties.from_values(
    top=460,
    bottom=450,
    initial_head=460,
    kx=8.64,
    ky=8.64,
    kz=0.864,
    specific_storage=1e-5,
    specific_yield=0.2
  )
)

project_command_bus.dispatch(create_model_layer_command)

In [7]:
model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
model.layers

LayersCollection(layers=[Layer(id=LayerId(value='7ac96bc6-0595-49e1-9471-d4a1aa67729e'), name=LayerName(value='Default'), description=LayerDescription(value='Default'), type=LayerType(type='confined'), properties=LayerProperties(kx=LayerPropertyValue(value=1.0, raster=None, zones=None), ky=LayerPropertyValue(value=1.0, raster=None, zones=None), kz=LayerPropertyValue(value=1.0, raster=None, zones=None), hani=None, vani=None, specific_storage=LayerPropertyValue(value=0.0001, raster=None, zones=None), specific_yield=LayerPropertyValue(value=0.1, raster=None, zones=None), initial_head=LayerPropertyValue(value=1.0, raster=None, zones=None), top=LayerPropertyValue(value=1.0, raster=None, zones=None), bottom=LayerPropertyValue(value=0.0, raster=None, zones=None))), Layer(id=LayerId(value='93659f2b-02c3-461d-8c3a-e28e14a8ae12'), name=LayerName(value='Top Layer'), description=LayerDescription(value='Top Layer'), type=LayerType(type='convertible'), properties=LayerProperties(kx=LayerPropertyValu

# Create Aquitard Layer

In [8]:
create_model_layer_command = CreateModelLayerCommand(
  project_id=project_id,
  model_id=model_id,
  user_id=user_id,
  layer_id=LayerId.new(),
  name=LayerName('Aquitard'),
  type=LayerType.confined(),
  description=LayerDescription('Aquitard'),
  properties=LayerProperties.from_values(
    top=450.0,
    bottom=448.0,
    initial_head=450.0,
    kx=10,
    ky=10,
    kz=0.1,
    specific_storage=1e-5,
    specific_yield=0.2,
  )
)

project_command_bus.dispatch(create_model_layer_command)

# Create Aquifer Layer

In [9]:
create_model_layer_command = CreateModelLayerCommand(
  project_id=project_id,
  model_id=model_id,
  user_id=user_id,
  layer_id=LayerId.new(),
  name=LayerName('Aquifer'),
  type=LayerType.confined(),
  description=LayerDescription('Aquifer'),
  properties=LayerProperties.from_values(
    top=448,
    bottom=390,
    initial_head=448,
    kx=8.64,
    ky=8.64,
    kz=0.864,
    specific_storage=1e-5,
    specific_yield=0.2,
  )
)

project_command_bus.dispatch(create_model_layer_command)

# Remove default layer

In [10]:
# get latest project layers

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)

layers = model.layers

# default layer id
default_layer_id = layers[0].id

# remove default layer
from morpheus.project.application.write.Model import DeleteModelLayerCommand

delete_model_layer_command = DeleteModelLayerCommand(
  project_id=project_id,
  model_id=model_id,
  user_id=user_id,
  layer_id=default_layer_id
)

project_command_bus.dispatch(delete_model_layer_command)

## Update Model Layer Name and Description and Type

In [11]:
from morpheus.project.application.write.Model import UpdateModelLayerCommand

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)

layers = model.layers

# bottom layer id
bottom_layer_id = layers[-1].id

update_model_layer_command = UpdateModelLayerCommand(
  project_id=project_id,
  model_id=model_id,
  user_id=user_id,
  layer_id=bottom_layer_id,
  layer_name=LayerName('Bottom Layer'),
  layer_description=LayerDescription('Bottom Layer'),
  layer_type=LayerType.unconfined()
)

project_command_bus.dispatch(update_model_layer_command)

model = model_reader.get_latest_model(project_id)
model.layers[-1]

Layer(id=LayerId(value='b6d7ff87-0dd4-435f-ae5e-0fdd6e750037'), name=LayerName(value='Bottom Layer'), description=LayerDescription(value='Bottom Layer'), type=LayerType(type='unconfined'), properties=LayerProperties(kx=LayerPropertyValue(value=8.64, raster=None, zones=None), ky=LayerPropertyValue(value=8.64, raster=None, zones=None), kz=LayerPropertyValue(value=0.864, raster=None, zones=None), hani=None, vani=None, specific_storage=LayerPropertyValue(value=1e-05, raster=None, zones=None), specific_yield=LayerPropertyValue(value=0.2, raster=None, zones=None), initial_head=LayerPropertyValue(value=448, raster=None, zones=None), top=LayerPropertyValue(value=448, raster=None, zones=None), bottom=LayerPropertyValue(value=390, raster=None, zones=None)))

## Update Model Layer Properties

In [12]:
from morpheus.project.application.write.Model import UpdateModelLayerPropertyCommand
from morpheus.project.types.layers.Layer import LayerPropertyValue, LayerPropertyName

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)

layers = model.layers

# bottom layer id
bottom_layer_id = layers[-1].id

update_model_layer_property_command = UpdateModelLayerPropertyCommand(
  project_id=project_id,
  model_id=model_id,
  user_id=user_id,
  layer_id=bottom_layer_id,
  property_name=LayerPropertyName.from_value('kx'),
  property_value=LayerPropertyValue.from_value(13.33)
)

project_command_bus.dispatch(update_model_layer_property_command)

model = model_reader.get_latest_model(project_id)
model.layers[-1].properties.kx

LayerPropertyValue(value=13.33, raster=None, zones=None)

# Show model version

In [13]:
model_reader = ModelReader()
model_reader.get_versions(project_id=project_id)


[ModelVersion(version_id=VersionId(value='10c63be6-8de6-416d-b48f-2e00455c597c'), tag=VersionTag(value='v0.0.0'), description=VersionDescription(value='Initial version'))]

In [14]:
model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
model.layers

LayersCollection(layers=[Layer(id=LayerId(value='93659f2b-02c3-461d-8c3a-e28e14a8ae12'), name=LayerName(value='Top Layer'), description=LayerDescription(value='Top Layer'), type=LayerType(type='convertible'), properties=LayerProperties(kx=LayerPropertyValue(value=8.64, raster=None, zones=None), ky=LayerPropertyValue(value=8.64, raster=None, zones=None), kz=LayerPropertyValue(value=0.864, raster=None, zones=None), hani=None, vani=None, specific_storage=LayerPropertyValue(value=1e-05, raster=None, zones=None), specific_yield=LayerPropertyValue(value=0.2, raster=None, zones=None), initial_head=LayerPropertyValue(value=460, raster=None, zones=None), top=LayerPropertyValue(value=460, raster=None, zones=None), bottom=LayerPropertyValue(value=450, raster=None, zones=None))), Layer(id=LayerId(value='eccf46a9-1585-46e0-b353-91fcc218955f'), name=LayerName(value='Aquitard'), description=LayerDescription(value='Aquitard'), type=LayerType(type='confined'), properties=LayerProperties(kx=LayerPropert